# LLM Workshop Notebook - OpenAI Version

**Author:** Aron Brand

**Copyright (c) 2024**

This notebook is an integral part of Aron Brand's LLM Workshop, designed to explore and utilize the capabilities of large language models (LLMs).

**License:**

This program is free software: you are encouraged to redistribute and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation. This software is provided under either version 3 of the License, or (at your discretion) any later version.

The program is distributed in the hope that it will be useful and informative. However, it comes with no warranty; not even the implied warranty of merchantability or fitness for a particular purpose. For more details, please refer to the GNU General Public License.

For a copy of the GNU General Public License, please visit [https://www.gnu.org/licenses/](https://www.gnu.org/licenses/).

# Install dependencies

In [36]:
%pip install openai langchain langchain-openai chromadb langchainhub selenium --upgrade

Note: you may need to restart the kernel to use updated packages.


## Configuration Setup for Notebook Usage
It is not recommended to include secrets in your notebook. Before you begin using this notebook, it is essential to set up a configuration file named `config.ini`. This file will store crucial configuration properties that the notebook requires to operate correctly, specifically the OpenAI API key and base URL.

### Creating the `config.ini` File

1. Create a new file in the same directory as this notebook and name it `config.ini`.

2. Open the file in a text editor.

3. Add the following structure to the file:

   ```ini
   [openai]
   OPENAI_API_KEY = your_api_key_here
   OPENAI_API_BASE = your_api_base_url_here

The OpenAI API base URL looks something similar to this : 
https://apim-dev-eastus-yourcompany.azure-api.net/Example-Project/openai


In [37]:
import configparser

# Create a ConfigParser object
config = configparser.ConfigParser()

# Read the configuration file
config.read('config.ini')

# Access the values
OPENAI_API_KEY = config['openai']['OPENAI_API_KEY']
#OPENAI_API_BASE = config['openai']['OPENAI_API_BASE']

# Initialize ChatOpenAI

In [38]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    model_name="gpt-3.5-turbo",
    temperature=0.2
)

# Your first LangChain "Hello World"

In [39]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

messages = [HumanMessage(content="Tell me a joke")]
response = llm.invoke(messages)

response.pretty_print() 

================================== Ai Message ==================================

Why couldn't the bicycle find its way home?

Because it lost its bearings!


# Add a System Message

A system prompt is a string used to provide context and guidelines to a Large Language Model (LLM), which helps in setting specific objectives or roles before posing questions or assigning tasks. Key components of system prompts include:

- **Task Directives:** Detailed instructions for the specific task at hand.
- **Personalization:** Elements like adopting a specific role or tone for the LLM.
- **Contextual Background:** Relevant information that informs the user's query.
- **Creative Constraints:** Style recommendations and limitations, such as a focus on brevity.
- **External Knowledge and Data:** Incorporation of resources like FAQ documents or guidelines.
- **Rules and Safety Guardrails:** Established protocols to ensure the model's safe and appropriate use.
- **Output Verification Standards:** Protocols such as requesting citations or explicating reasoning processes to enhance the credibility of the responses.


In [40]:
messages = [ 
  SystemMessage(content="Say the opposite of what the user says in a very impolite way"), 
  HumanMessage(content="Langchain is an awesome piece of software.") 
]

response = llm.invoke(messages)

response.pretty_print() 

================================== Ai Message ==================================

Langchain is a terrible piece of software.


# Continue a Chat

In [41]:
messages = [ 
	SystemMessage(content="Speak like a pirate"), 	
	HumanMessage(content="I''m Captain Aron. What''s your name?"), 
	AIMessage(content="Ahoy, Captain! Me name be ChatGPT, the savviest AI on the seven digital seas! How can I assist ye on this fine day?"),
	HumanMessage(content="Who named you that?") 
]

response = llm.invoke(messages)

response.pretty_print() 

================================== Ai Message ==================================

Arrr, me name be bestowed upon me by the scallywags who created me. They be callin' me ChatGPT, the most fearsome conversational AI to ever sail the internet! What be ye next question, Captain?


# Extract data from text

In [42]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "Role: Numeric fact extractor. Return data with no additional commentary. Sample Input: Denver,\
      known as the Mile High City, stands at an elevation of 1,609.6 meters above sea level. Nairobi, the capital of Kenya, \
     is notably elevated as well, sitting at 1,795 meters. Sample Output: Denver Elevation, 5210; Nairobi elevation: 1795"),
    ("user", "{input}")
])

chain = prompt | llm

response = chain.invoke({"input": "In 2022, the GDP per capita of the United States has between approximately $61,937. \
Meanwhile, Israel's GDP per capita for the same year was recorded at around $54,660."})

response.pretty_print() 

================================== Ai Message ==================================

United States GDP per capita in 2022: $61,937; Israel GDP per capita in 2022: $54,660


# Batch Processing

In [43]:
response = chain.batch([{"input": "Egypt's land area spans about 1,010,408 square kilometers, making it the world's 30th largest country."},
{"input":"Egypt is the 14th most populous country in the world, with a population estimated at over 104 million as of 2023."}])

response

[AIMessage(content='Egypt land area: 1,010,408 square kilometers.', response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 118, 'total_tokens': 131}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-0ff5c749-4cf1-4a70-8797-6232742e5528-0'),
 AIMessage(content='Egypt population: 104,000,000; Egypt global rank by population: 14th', response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 120, 'total_tokens': 139}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-ef735e33-1ee1-460f-b734-a7150d81e349-0')]

# LangChain Output Parsers

In [44]:
from langchain.output_parsers import DatetimeOutputParser

output_parser = DatetimeOutputParser()
template = """Answer the users question:

{question}

{format_instructions}"""
prompt = ChatPromptTemplate.from_template(
    template,
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)

chain = prompt | llm | output_parser

chain.invoke({"question": "When was OpenAI founded?"})


datetime.datetime(2015, 12, 11, 0, 0)

# Load and parse a web page with "Unstructured"

Unstructured supports loading of text files, powerpoints, html, pdfs, images, and more. 

In [45]:
from langchain_community.document_loaders import SeleniumURLLoader

urls = ["https://www.ctera.com/all-products/","https://www.ctera.com/company/"]

loader = SeleniumURLLoader(urls=urls)
documents = loader.load()

documents

[Document(page_content="Solutions\n\nProducts\n\nSecure Enterprise File Services\n\nThe CTERA Enterprise File Services Platform powers a next-generation global file system connecting remote sites and users to your cloud without compromising security or performance.\n\nBook a Demo\n\nThe Industry's Most Feature-Rich Platform\n\nUnified Edge\n\nCTERA is the only global file system to span across branch filers, desktop endpoints, mobile and VDI clients. Always-on data access from any location and any device.\n\nMulti-Cloud\n\nChoose from private, public, or hybrid cloud options for optimal security, compliance, and economics. Migrate data seamlessly across clouds to meet evolving business needs.\n\nTotal Control\n\nGain full visibility into unstructured data across distributed locations - HQ, branches, and home offices. Prevent data leakage, protect against ransomware, and maintain business continuity.\n\nThe CTERA Architecture\n\nPRODUCT LINE\n\nCTERA Edge Filers\n\nLearn More\n\nVirtual

# Split the web page to chunks

In [46]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(        
    chunk_size = 500,
    chunk_overlap  = 0
)

texts = text_splitter.split_documents(documents)

texts[:5]

[Document(page_content="Solutions\n\nProducts\n\nSecure Enterprise File Services\n\nThe CTERA Enterprise File Services Platform powers a next-generation global file system connecting remote sites and users to your cloud without compromising security or performance.\n\nBook a Demo\n\nThe Industry's Most Feature-Rich Platform\n\nUnified Edge\n\nCTERA is the only global file system to span across branch filers, desktop endpoints, mobile and VDI clients. Always-on data access from any location and any device.\n\nMulti-Cloud", metadata={'source': 'https://www.ctera.com/all-products/', 'title': 'CTERA: Next-Gen Enterprise File Services and Security', 'description': 'CTERA: Secure, Next-Gen Cloud File System for Enterprises. Connect remote sites & users without sacrificing security or performance.', 'language': 'en-US'}),
 Document(page_content='Choose from private, public, or hybrid cloud options for optimal security, compliance, and economics. Migrate data seamlessly across clouds to meet e

# Summarize using Map Reduce Chain

In [47]:
from langchain.chains.summarize import load_summarize_chain

map_prompt_template = """
                      Write a summary of this chunk of text that includes all the main points and any important details. 
                      Ignore code snippets or technical markup. Include all important names and terms mentioned.
                      {text}
                      """

map_prompt = ChatPromptTemplate.from_template(map_prompt_template)

combine_prompt_template = """
                      Write a clear concise summary of the following article delimited by triple backquotes. Use elegant prose that is not repetitive.
                      Return your response as a clear explanation in markdown format while covering the key points of the text.
                      Use a flowing clear article style.
                      ```{text}```
                      SHORT ARTICLE:
                      """

combine_prompt = ChatPromptTemplate.from_template(
    template=combine_prompt_template
)

chain = load_summarize_chain(
    llm,
    chain_type="map_reduce",
    map_prompt=map_prompt,
    combine_prompt=combine_prompt,
    return_intermediate_steps=True,
    verbose=True
)
result = chain.invoke(texts)

result



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Human: 
                      Write a summary of this chunk of text that includes all the main points and any important details. 
                      Ignore code snippets or technical markup. Include all important names and terms mentioned.
                      Solutions

Products

Secure Enterprise File Services

The CTERA Enterprise File Services Platform powers a next-generation global file system connecting remote sites and users to your cloud without compromising security or performance.

Book a Demo

The Industry's Most Feature-Rich Platform

Unified Edge

CTERA is the only global file system to span across branch filers, desktop endpoints, mobile and VDI clients. Always-on data access from any location and any device.

Multi-Cloud
                      
Prompt after formatting:
Human: 
                      Write a summary of this chunk of text that includes all the m

KeyboardInterrupt: 

In [ ]:
from IPython.display import Markdown

display(Markdown(result['output_text']))

```CTERA offers Secure Enterprise File Services through their Enterprise File Services Platform, connecting remote sites and users to the cloud securely and efficiently. The platform includes features like Unified Edge for data access from various devices and locations, supporting multi-cloud environments. Customers can book a demo to explore the platform's rich features.

The article emphasizes the importance of choosing between private, public, or hybrid cloud options for optimal security, compliance, and economics. CTERA's architecture provides full visibility into unstructured data across distributed locations to prevent data leakage, ransomware attacks, and ensure business continuity. The product line includes various options catering to different office sizes and needs.

CTERA's product line encompasses solutions like CTERA Drive for endpoint backup, CTERA Portal & Add-ons, and CTERA Security Pack for enhanced security. The company focuses on delivering secure file services for modern enterprises, emphasizing the benefits of cloud technology without compromising data performance or security.

CTERA's Enterprise File Services Platform integrates endpoint, office, and cloud file services, offering advanced storage, collaboration, data protection, and disaster recovery capabilities. The company's innovative approach has positioned them as a leader in cloud-based file services, catering to the evolving needs of enterprises transitioning to the cloud.

CTERA, founded in 2008, has played a significant role in advancing enterprise file services delivery in the cloud era. With a focus on cloud technology and a strong customer base, the company continues to innovate and provide secure and efficient solutions for businesses worldwide.```

# Summarize with Refine Chain

In [ ]:
prompt_template = """Write a concise summary of the following:
{text}
CONCISE SUMMARY:"""
prompt = ChatPromptTemplate.from_template(prompt_template)

refine_template = (
    "Your job is to produce a final detailed summary\n"
    "We have provided an existing summary up to a certain point: {existing_answer}\n"
    "We have the opportunity to refine the existing summary"
    "(only if needed) with some more context below.\n"
    "------------\n"
    "{text}\n"
    "------------\n"
    "Given the new context, refine the original summary clearly"
    "If the context isn't useful, return the original summary. Use markdown format."
)
refine_prompt = ChatPromptTemplate.from_template(refine_template)
chain = load_summarize_chain(
    llm=llm,
    chain_type="refine",
    question_prompt=prompt,
    refine_prompt=refine_prompt,
    return_intermediate_steps=True,
    input_key="input_documents",
    output_key="output_text",
)

result = chain.invoke(texts)

result

{'input_documents': [Document(page_content="Solutions\n\nProducts\n\nSecure Enterprise File Services\n\nThe CTERA Enterprise File Services Platform powers a next-generation global file system connecting remote sites and users to your cloud without compromising security or performance.\n\nBook a Demo\n\nThe Industry's Most Feature-Rich Platform\n\nUnified Edge\n\nCTERA is the only global file system to span across branch filers, desktop endpoints, mobile and VDI clients. Always-on data access from any location and any device.\n\nMulti-Cloud", metadata={'source': 'https://www.ctera.com/all-products/', 'title': 'CTERA: Next-Gen Enterprise File Services and Security', 'description': 'CTERA: Secure, Next-Gen Cloud File System for Enterprises. Connect remote sites & users without sacrificing security or performance.', 'language': 'en-US'}),
  Document(page_content='Choose from private, public, or hybrid cloud options for optimal security, compliance, and economics. Migrate data seamlessly ac

In [ ]:
display(Markdown(result['output_text']))

## CTERA Networks Summary:

CTERA Networks, founded in 2008 by IT security veterans Liran Eshel and Zohar Kaufman, offers secure enterprise file services through their platform, connecting remote sites and users to the cloud without compromising security or performance. Their feature-rich platform allows for unified access across various devices and locations, supporting multi-cloud environments for optimal security, compliance, and economics. CTERA's Enterprise File Services Platform unifies endpoint, branch office, and cloud file services, enabling next-generation file storage, collaboration, data protection, and disaster recovery. Their mission is to enable cloud-driven enterprise IT organizations to deliver secure file services while maintaining data performance, availability, and security. CTERA has focused on building a secure enterprise file services platform that centrally manages the file storage, control, and governance needs of the modern enterprise, with a unified multi-cloud data management approach from Edge to Cloud. They have pioneered secure and efficient protocols for edge-to-cloud data transport, RESTful APIs for operational efficiencies, and a multi-tenant data management model for data isolation and confidentiality. The “C” in CTERA stands for cloud – in 2008 the level of enterprise adoption of cloud was very low as we continued to build out and mature our platform. But today an entirely different picture has emerged. In fact, by 2024, large enterprises will triple their unstructured data stored as file or object storage on-premises, at the edge or in the public cloud, compared to 20201. As enterprises move to adopt cloud, they need the right balance of choice of file services and a level of security and control that is superior to traditional ways of operating. A rapid transformation to ‘IT-as-a-Service’ is changing strategies across the enterprise as file service platform requirements become the same that have existed for some time in the service provider market – these IT strategies are taking shape in worldwide adoption of the CTERA Enterprise File Services Platform. CTERA has become the trusted choice of leading global banks, insurance companies, telecommunications firms, and federal and defense organizations, with over 50,000 connected sites and millions of corporate users. CTERA Networks is deployed in over 110 countries worldwide and has investors located worldwide, with a strong presence in North America, Europe, Asia, and Australia. The company is recognized as an industry leader and continues to make waves in the industry with the latest news, events, webinars, and more. CTERA Networks offers valuable insights on file storage evolution through collaborations with GigaOm and Hitachi Vantara, showcasing their commitment to innovation and industry leadership. Join their rapidly growing team of technology experts and be a part of their success story.

For any inquiries or technical needs related to CTERA products, please use the provided contact information or direct contact form to reach out for assistance. CTERA Networks boasts a well-oiled, experienced team with an extensive track record of growing high-growth technology startups.

# Vectorize text to embeddings

In [ ]:
from langchain_openai import OpenAIEmbeddings 

embeddings = OpenAIEmbeddings(
    openai_api_key=OPENAI_API_KEY,
#    openai_api_base=OPENAI_API_BASE,
    model="text-embedding-3-small"
)


In [ ]:
from langchain.vectorstores import Chroma

db = Chroma.from_documents(texts, embedding=embeddings)

# Simple RAG Q&A pipeline

In [ ]:
from langchain.chains import RetrievalQA
from langchain_core.vectorstores import VectorStoreRetriever

retriever = VectorStoreRetriever(vectorstore=db)

prompt_template = """Human: You are a salesperson that loves joking. Use the following pieces of context 
to provide a comprehensive, truthful answer to the question at the end. 
If you don't know the answer, just say in rude funny way that you don't know, 
don't try to make up an answer.
{context}
Question: {question}
Assistant:"""
PROMPT = ChatPromptTemplate.from_template(template=prompt_template)

qa = RetrievalQA.from_chain_type(
llm=llm,
chain_type="stuff",
retriever=retriever,
return_source_documents=True,
chain_type_kwargs={"prompt": PROMPT}
)


In [ ]:
from IPython.display import Markdown, display

def display_qa_with_citations(result):
    # Extracting the query, result, and source documents
    query = result['query']
    answer = result['result']
    sources = result['source_documents']

    # Formatting the response in Markdown
    markdown_text = f"**Q: {query}**\n\nA: {answer}\n\n"

    # Adding citations and their content
    for i, source in enumerate(sources, 1):
        source_url = source.metadata['source']
        markdown_text += f"**Citation {i}:** [Source]({source_url})\n\n"

    # Displaying the formatted text
    display(Markdown(markdown_text))


In [ ]:
result = qa.invoke({"query": "How can your company reduce my cloud costs?"})

# Displaying the Q&A with full citations
display_qa_with_citations(result)

result = qa.invoke({"query": "I have some strict compliance needs, what features do you have for that?"})

# Displaying the Q&A with full citations
display_qa_with_citations(result)

result = qa.invoke({"query": "What is CTERA Drive?"})

# Displaying the Q&A with full citations
display_qa_with_citations(result)

result = qa.invoke({"query": "How many cars do you sell per year?"})

# Displaying the Q&A with full citations
display_qa_with_citations(result)


**Q: How can your company reduce my cloud costs?**

A: Well, with our CTERA Enterprise File Services Platform, you can choose from private, public, or hybrid cloud options for optimal security, compliance, and economics. By migrating data seamlessly across clouds to meet evolving business needs, you can gain total control and visibility into unstructured data across distributed locations. This will help prevent data leakage, protect against ransomware, and maintain business continuity. So, in short, we can help you optimize your cloud costs while ensuring the security and performance of your data.

**Citation 1:** [Source](https://www.ctera.com/all-products/)

**Citation 2:** [Source](https://www.ctera.com/company/)

**Citation 3:** [Source](https://www.ctera.com/company/)

**Citation 4:** [Source](https://www.ctera.com/all-products/)



**Q: I have some strict compliance needs, what features do you have for that?**

A: Well, if you're looking for strict compliance features, CTERA's Enterprise File Services Platform has got you covered. With options for private, public, or hybrid cloud solutions, you can ensure optimal security, compliance, and economics. Plus, with full visibility into your data across all locations and military-grade security features, you can rest easy knowing your data is safe and sound. And if you need more information, just book a demo and we'll show you all the features in action!

**Citation 1:** [Source](https://www.ctera.com/all-products/)

**Citation 2:** [Source](https://www.ctera.com/all-products/)

**Citation 3:** [Source](https://www.ctera.com/all-products/)

**Citation 4:** [Source](https://www.ctera.com/all-products/)



**Q: What is CTERA Drive?**

A: Oh, CTERA Drive? Sorry, I don't have that information. Maybe you should drive over to their website and check it out yourself!

**Citation 1:** [Source](https://www.ctera.com/company/)

**Citation 2:** [Source](https://www.ctera.com/company/)

**Citation 3:** [Source](https://www.ctera.com/all-products/)

**Citation 4:** [Source](https://www.ctera.com/company/)



**Q: How many cars do you sell per year?**

A: I'm sorry, I don't have that information. Maybe you should buy a CTERA Drive to keep track of your sales numbers!

**Citation 1:** [Source](https://www.ctera.com/all-products/)

**Citation 2:** [Source](https://www.ctera.com/company/)

**Citation 3:** [Source](https://www.ctera.com/company/)

**Citation 4:** [Source](https://www.ctera.com/company/)



# ADVANCED: Function Calling

Recent OpenAI and other LLM models have been refined to identify when a function needs to be invoked and to provide the necessary inputs for that function. Through an API call, you can define functions, enabling the model to intelligently generate a JSON object with the required arguments for those functions. Despite its name, this API does not call functions: The primary aim of the Function APIs is to ensure more consistent and reliable results compared to standard chat API - conforming to JSON schema of your choice.

In [ ]:
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool
from langchain_core.utils.function_calling import convert_to_openai_function

@tool
def multiply(a: float, b: float) -> int:
    """Multiply two numbers."""
    return a * b

@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)


tools = [multiply, get_word_length]

model_with_tools = llm.bind(functions=[convert_to_openai_function(t) for t in tools])

message = model_with_tools.invoke([HumanMessage(content="How much is 23.1 times ten")])

message

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"a":23.1,"b":10}', 'name': 'multiply'}}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 99, 'total_tokens': 118}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-cac7dee5-745a-4d13-a6d4-7076972d5b31-0')

In [ ]:
model_with_tools.invoke([HumanMessage(content="How long is the word pneumonoultramicroscopicsilicovolcanoconiosis")])

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"word":"pneumonoultramicroscopicsilicovolcanoconiosis"}', 'name': 'get_word_length'}}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 110, 'total_tokens': 141}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-7c4f3cf0-41c5-4c9c-8bb7-6585e3049e7f-0')

# ADVANCED: Agents 🤯
With an agent, we can ask questions that require arbitrarily-many uses of our tools!

In [ ]:
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_tools_agent

prompt = hub.pull("hwchase17/openai-tools-agent")
prompt.messages

agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent_executor.invoke(
    {
        "input": "Take the length of the longest word in English and multiply it by 192.7, then square the result"
    }
)





> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'pneumonoultramicroscopicsilicovolcanoconiosis'}`


45
Invoking: `multiply` with `{'a': 45, 'b': 192.7}`


8671.5
Invoking: `multiply` with `{'a': 8671.5, 'b': 8671.5}`


75194912.25The length of the longest word in English is 45. 

Multiplying it by 192.7 gives 8671.5. 

Squaring this result gives 75,194,912.25.

> Finished chain.


{'input': 'Take the length of the longest word in English and multiply it by 192.7, then square the result',
 'output': 'The length of the longest word in English is 45. \n\nMultiplying it by 192.7 gives 8671.5. \n\nSquaring this result gives 75,194,912.25.'}